# One-time pad adventures

## Encryption and Decryption

Let's start by creating some fake "truly" random bytes.

In [3]:
from Crypto.Random import get_random_bytes

padBytes = get_random_bytes(100)

These bytes are pre-computed ahead of time and shared between the people sending messages between each other. They must remain secret forever.

Now let's define a message

In [4]:
plaintext = b'Kill the king at midnight'

Let's define the encryption / decryption function.

In [11]:
def encrypt_decrypt(payload, key):
    output = bytearray()
    for i in range(0, len(payload)):
        outputByte = payload[i] ^ key[i]
        output.append(outputByte)
    return output

And test:

In [12]:
cipherText = encrypt_decrypt(plaintext, padBytes )
cipherText

bytearray(b'q\xbeP@\xfd\x7fG\x9a\x92\x1c\x82E\x13\x98\x06\x01H\x18+\x9c\r\xda\xf8Z\xda')

And check that we can recover the plain-text:

In [14]:
encrypt_decrypt(cipherText, padBytes)

bytearray(b'Kill the king at midnight')

## Attack: Using the same key twice

Using the same key twice completely destroys security. Suppose I have two messages $m_0$ and $m_1$. They are the same length and encrypted with the same key, $k$.

Then by commutativity of XOR and the property that $\forall A: A \oplus A = 0$:

$$(m_0 \oplus  k) \oplus  (m_1 \oplus  k) \\ = (m_0 \oplus  m_1) \oplus  (k \oplus  k) \\ = (m_0 \oplus  m_1)  $$

The key cancels out and we're left with the logical XOR of the two plain-texts. If we know one of them, we can trivially compute the other.

Even if you don't know either of them, you can recover both texts by looking at where the zeros are. Everywhere you have a matching letter in the same position, they 
will cancel each other out and we'll end up with a zero. 

"e" is the most common English letter. So "e" is the mostly likely to overlap between messages. Solve by filling in the letters like a Sudoku puzzle. 

Even though this full plain-text recovery attack exists, let's show there's a problem using our semantic security framework!

In [22]:
class OTP_ReusedKey:
    def __init__(self):
        self.key = get_random_bytes(100)

    def encrypt(self, data):
        return encrypt_decrypt(data, self.key)         


{'Trials': 200, 'Wins': 200}

In the above snippet, we fix the key at the start of the game and just repeatedly encrypt with the same key.

The strategy below is the same as what we saw with RC4. On the first trial, send the same message twice to get the encryption.

On all other attempts, send the pair. Just examine which one comes back.

In [ ]:
class OTP_ReusedKey_Strategy:
    def getMessages(self, trialNumber):
        m0 = b'Kill the king at 2am'
        m1 = b'Kill the king at 3am'

        if trialNumber == 0:
            return m0, m0
        else:
            return m0, m1 

    def challenge(self, challenge, trialNumber):
        if trialNumber == 0:
            self.m0Encrypted = challenge
            return 0
        else:
            if challenge == self.m0Encrypted:
                return 0
            else:
                return 1

In [ ]:
from  SemanticSecurityGame import SemanticSecurityGame
game = SemanticSecurityGame(OTP_ReusedKey(), OTP_ReusedKey_Strategy())
game.runGame(200)